# Building Intelligent Agents with DeepSeek R1, CrewAI and SageMaker

This notebook demonstrates how to build and deploy an intelligent agent system using DeepSeek R1 (Distilled), CrewAI and Amazon SageMaker. We'll create a multi-agent system called ScribbleBots that can perform research and writing tasks using state-of-the-art language models.

## Overview
1. Set up the development environment
2. Deploy DeepSeek R1 LLaMA 3.3 70B model on SageMaker
3. Create intelligent agents using CrewAI
4. Implement a sequential workflow for research and writing tasks

## Prerequisites
- AWS Account with SageMaker access
- Hugging Face account and API token
- Python 3.8+

## Step 1: Environment Setup

First, let's install the required dependencies. We'll need CrewAI for agent orchestration, boto3 for AWS interactions, and various ML-related packages.

In [ ]:
%pip install -qU crewai boto3 sagemaker "streamlit==1.38.0" huggingface_hub psutil pynvml

In [ ]:
%pip install --upgrade transformers==4.44.2 torch>=1.1.13 torchvision torchaudio

## Step 2: Configure Environment Variables

Set up the necessary credentials and configuration for AWS and Hugging Face. Make sure to replace the empty strings with your actual values.

In [ ]:
## Configuration Variables
bucket_name = ""  # Your S3 bucket name
HUGGING_FACE_HUB_TOKEN = ""  # Your Hugging Face token
my_region_name = ""  # Your AWS region (e.g., "us-east-1")

# Validation
assert bucket_name != "", "Please provide Bucket name above"
assert HUGGING_FACE_HUB_TOKEN != "", "Please provide Huggingface token above"
assert my_region_name != "", "Please provide a region above"

## Step 3: Deploy and Predict Using DeepSeek R1 Distilled Model on SageMaker

We'll deploy the DeepSeek R1 Distilled LLaMA 3.3 70B model on SageMaker. This model will serve as our primary language model for the agents.

In [ ]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
import os
from datetime import datetime

# Configure AWS and SageMaker
number_of_gpu = 8
boto_session = boto3.Session(region_name=my_region_name)
sagemaker_session = sagemaker.Session(boto_session=boto_session, default_bucket=bucket_name)

# Get or create IAM role
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Model configuration
hub = {
	'HF_MODEL_ID':'deepseek-ai/DeepSeek-R1-Distill-Llama-70B', #Llama-3.3-70B-Instruct
	'SM_NUM_GPUS': json.dumps(number_of_gpu),
    'HF_TOKEN': HUGGING_FACE_HUB_TOKEN,
    'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',  # Set to INFO level
    'PYTORCH_CUDA_ALLOC_CONF': 'expandable_segments:True'  # Add this line
}


# Create endpoint name
custom_endpoint_name = f'deepseek-r1-dist-v3-llama70b-{datetime.now().strftime("%Y-%m-%d")}'

# Create and deploy model
huggingface_model = HuggingFaceModel(
    image_uri=get_huggingface_llm_image_uri("huggingface", version="2.3.1"),
    env=hub,
    role=role,
    sagemaker_session=sagemaker_session
)

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.p4d.24xlarge",
    endpoint_name=custom_endpoint_name,
    container_startup_health_check_timeout=900
)

# Test the endpoint
response = predictor.predict({
    "inputs": "Hi, what can you help me with?"
})
print("Test response:", response)

## Step 4: Create ScribbleBots - An Agentic Research Writer

Now we'll create a multi-agent system consisting of a Research Agent and a Writer Agent. These agents will work together in a sequential workflow to research topics and create high-quality written content.

In [41]:
import os
from crewai import Crew, Agent, Task, Process
import sys
sys.path.append(os.path.dirname(os.getcwd()))
from tools.sage_tools import CustomTool, deepseek_llama_inference, DeepSeekSageMakerLLM


# Create DeepSeek inference tool
deepseek_tool = CustomTool(
    name="deepseek_llama_3.3_70B",
    func=lambda inputs: deepseek_llama_inference(
        prompt=inputs,
        endpoint_name=custom_endpoint_name
    ),
    description="A tool to generate text using the DeepSeek LLaMA model deployed on SageMaker."
)

### Quick Note

If you see this error, please diregard it




<img src="./images/error.png" alt="workflow diagram" style="border: 5px solid black; padding: 10px; box-shadow: 5px 5px 5px grey;">

### Create Research Agent

The Research Agent is responsible for gathering and analyzing information from various sources.

In [ ]:
research_agent = Agent(
    role="Research Bot",
    goal="Scan sources, extract relevant information, and compile a research summary.",
    backstory="An AI agent skilled in finding relevant information from a variety of sources.",
    tools=[deepseek_tool],
    allow_delegation=True,
    llm=DeepSeekSageMakerLLM(endpoint=custom_endpoint_name),
    verbose=False
)

### Create Writer Agent

The Writer Agent transforms research findings into polished, structured content.

In [ ]:
writer_agent = Agent(
    role="Writer Bot",
    goal="Receive research summaries and transform them into structured content.",
    backstory="A talented writer bot capable of producing high-quality, structured content based on research.",
    tools=[deepseek_tool],
    allow_delegation=False,
    llm=DeepSeekSageMakerLLM(endpoint=custom_endpoint_name),
    verbose=False
)

### Define Agent Tasks

Set up the specific tasks for each agent in the workflow.

In [ ]:
research_task = Task(
    description=(
        "Your task is to conduct research based on the following query: {prompt}.\n"
        "- Scan multiple sources to gather relevant information.\n"
        "- Summarize your findings into a concise, well-organized research summary."
    ),
    expected_output="A comprehensive research summary based on the provided query.",
    agent=research_agent,
    tools=[deepseek_tool]
)

writing_task = Task(
    description=(
        "Your task is to create structured content based on the research provided.\n"
        "- Transform the research into high-quality written content.\n"
        "- Ensure the output is clear, organized, and adheres to professional writing standards.\n"
        "- Focus on maintaining the key points while improving readability and flow."
    ),
    expected_output="A well-structured article based on the research summary.",
    agent=writer_agent,
    tools=[deepseek_tool]
)

### Create and Execute the Crew

Finally, we'll create the crew with our agents and tasks, then execute a test workflow.

# Building Agents with CrewAI and SageMaker

## Execute Research Workflow
Now we'll execute our research and writing workflow with the configured agents.

In [ ]:
scribble_bots = Crew(
    agents=[research_agent, writer_agent],
    tasks=[research_task, writing_task],
    process=Process.sequential  # Ensure tasks execute in sequence
)

In [ ]:
def format_response(raw_text):
    """Format the raw response text for better readability"""
    formatted_text = raw_text.replace('\\n', '\n').strip()
    return formatted_text

def parse_agent_response(response):
    """Helper function to parse and structure agent responses"""
    thought = "Could not extract Chain of Thought."
    answer = "Could not extract Final Answer."
    
    try:
        if isinstance(response, str):
            response = format_response(response)
            
            if "Thought:" in response:
                thought_start = response.find("Thought:") + len("Thought:")
                thought_end = response.find("'Final Answer':") if "'Final Answer':" in response else len(response)
                thought = response[thought_start:thought_end].strip()
            
            if "'Final Answer':" in response:
                answer_start = response.find("'Final Answer':") + len("'Final Answer':")
                answer = response[answer_start:].strip()
                
    except Exception as e:
        print(f"Error parsing response: {e}")
        
    return thought, answer

# Execute the workflow
print("Starting ScribbleBots Sequential Workflow...")
result = scribble_bots.kickoff(inputs={"prompt": "Explain what is deepseek very briefly"})

# Parse and display results
if hasattr(result, 'raw') and isinstance(result.raw, dict):
    raw_response = result.raw
    if isinstance(raw_response, list) and "generated_text" in raw_response[0]:
        thought, answer = parse_agent_response(raw_response[0]["generated_text"])
    else:
        thought, answer = parse_agent_response(str(raw_response))
else:
    thought, answer = parse_agent_response(str(result))

# Display parsed outputs
print("***************************************")
print("\n=== Chain of Thought ===")
print("***************************************")
print(thought)
print("***************************************")
print("***************************************")
print("\n=== Final Answer ===")
print(answer)
print("***************************************")
print("***************************************")